In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Rio Grande Do Norte - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_RN.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Rio Grande Do Norte - Produção de Cimento (t)',axis=1)
data

,Ano,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande Do Norte - Consumo de Cimento (t)
0,2008,3.355987e+07,2.079113e+06,9.184161,2.924771e+07,0.817253,0.651662,8.054622,5.483335e+08,2.546644e+09,...,0.667512,0.709150,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682,705.791000
1,2009,3.376729e+07,2.091124e+06,9.172749,2.923115e+07,0.819840,0.655238,8.084471,5.302764e+08,2.679945e+09,...,0.669561,0.711755,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049,694.478000
2,2010,3.462025e+07,2.094395e+06,9.216787,2.947144e+07,0.821734,0.658498,8.047148,5.046425e+08,2.891930e+09,...,0.671269,0.713945,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629,850.190000
3,2011,3.538151e+07,2.094876e+06,9.249340,2.964338e+07,0.822949,0.661438,8.031985,4.854622e+08,3.091988e+09,...,0.672647,0.715731,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734,836.975000
4,2012,3.604796e+07,2.083213e+06,9.259759,2.976380e+07,0.823515,0.663198,12.625418,4.867661e+08,3.189506e+09,...,0.673384,0.716652,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622,961.431000
5,2013,3.658711e+07,2.061661e+06,9.253188,2.982187e+07,0.823225,0.667048,12.889274,5.225848e+08,3.093872e+09,...,0.674666,0.718456,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194,954.170000
6,2014,3.698650e+07,2.024778e+06,9.243102,2.982142e+07,0.822577,0.669135,13.251300,5.676413e+08,2.779340e+09,...,0.675055,0.719161,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147,886.165714
7,2015,3.730024e+07,1.995627e+06,9.238580,2.982594e+07,0.821459,0.669230,13.501882,5.179392e+08,2.561731e+09,...,0.674976,0.718841,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726,802.508571
8,2016,3.749595e+07,1.966808e+06,9.235352,2.982092e+07,0.834707,0.679146,13.466118,4.524144e+08,2.516859e+09,...,0.683189,0.729101,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474,730.261714
9,2017,3.759092e+07,1.955499e+06,9.243656,2.984006e+07,0.833862,0.679432,13.134883,4.213689e+08,2.497710e+09,...,0.682843,0.728837,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241,646.482000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,3.355987e+07,2.079113e+06,9.184161,2.924771e+07,0.817253,0.651662,8.054622,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,0.667512,0.709150,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682
1,3.376729e+07,2.091124e+06,9.172749,2.923115e+07,0.819840,0.655238,8.084471,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,0.669561,0.711755,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049
2,3.462025e+07,2.094395e+06,9.216787,2.947144e+07,0.821734,0.658498,8.047148,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,0.671269,0.713945,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629
3,3.538151e+07,2.094876e+06,9.249340,2.964338e+07,0.822949,0.661438,8.031985,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,0.672647,0.715731,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734
4,3.604796e+07,2.083213e+06,9.259759,2.976380e+07,0.823515,0.663198,12.625418,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,0.673384,0.716652,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622
5,3.658711e+07,2.061661e+06,9.253188,2.982187e+07,0.823225,0.667048,12.889274,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,0.674666,0.718456,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194
6,3.698650e+07,2.024778e+06,9.243102,2.982142e+07,0.822577,0.669135,13.251300,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,0.675055,0.719161,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147
7,3.730024e+07,1.995627e+06,9.238580,2.982594e+07,0.821459,0.669230,13.501882,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,0.674976,0.718841,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726
8,3.749595e+07,1.966808e+06,9.235352,2.982092e+07,0.834707,0.679146,13.466118,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,0.683189,0.729101,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474
9,3.759092e+07,1.955499e+06,9.243656,2.984006e+07,0.833862,0.679432,13.134883,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,0.682843,0.728837,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     694.478000
1     850.190000
2     836.975000
3     961.431000
4     954.170000
5     886.165714
6     802.508571
7     730.261714
8     646.482000
9     485.012000
10    563.293000
11    744.863000
12    811.860000
13    866.416000
14           NaN
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.831952,0.851916,-1.776864,-1.942366,-1.428923,-1.679355,-1.408085,1.204752,-0.990234,0.166091,...,0.984445,-1.525355,-1.569868,2.457245,-1.212494,1.862951,-0.705394,-0.762192,-0.756764,1.972247
1,-1.686476,1.061967,-2.221110,-2.019038,-0.974260,-1.304896,-1.395638,0.820657,-0.546688,0.428911,...,-0.051710,-1.159167,-1.188723,0.144226,-1.582145,1.855564,-0.347758,-0.699954,-0.781667,1.489960
2,-1.088236,1.119167,-0.506765,-0.906710,-0.641303,-0.963539,-1.411202,0.275392,0.158672,0.847995,...,-0.115145,-0.853862,-0.868295,-0.568350,-1.518190,1.208644,-1.439934,-1.053053,-0.893344,1.034020
3,-0.554313,1.127579,0.760462,-0.110759,-0.427758,-0.655663,-1.417525,-0.132596,0.824346,1.230291,...,0.633403,-0.607606,-0.606843,-0.695505,-0.774675,0.248067,-1.287958,-1.257539,-1.198046,0.513803
4,-0.086886,0.923630,1.166077,0.446668,-0.328271,-0.471367,0.497973,-0.104860,1.148829,1.382743,...,-0.646476,-0.475880,-0.472170,0.071997,-0.284967,-0.279787,-0.574331,-1.010638,-1.104686,0.113084
5,0.291258,0.546720,0.910252,0.715452,-0.379303,-0.068230,0.608003,0.657046,0.830615,1.112297,...,-0.757614,-0.246760,-0.208132,1.159926,0.371328,-0.502457,-0.529965,-0.573928,-0.578367,-0.299977
6,0.571377,-0.098301,0.517622,0.713363,-0.493169,0.150343,0.758971,1.615453,-0.215960,0.282052,...,0.345772,-0.177266,-0.105014,0.479702,1.187601,-0.630330,0.031926,-0.036991,-0.079642,-0.497452
7,0.791421,-0.608079,0.341588,0.734301,-0.689751,0.160247,0.863466,0.558230,-0.940033,-0.632425,...,1.298413,-0.191340,-0.151836,-0.445543,1.261073,-0.699772,0.993697,0.483365,0.289839,-0.481495
8,0.928688,-1.112071,0.215918,0.711070,1.638563,1.198697,0.848551,-0.835563,-1.089341,-1.093812,...,1.596124,1.276482,1.349626,-1.105884,0.913840,-0.740852,0.416771,0.706397,0.759507,-0.565156
9,0.995293,-1.309848,0.539207,0.799679,1.490090,1.228649,0.710424,-1.495939,-1.153056,-1.335633,...,-0.048438,1.214568,1.311056,-1.310433,0.799449,-0.768068,0.215438,1.012997,1.226895,-0.874702


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.74079602, -1.11662735, -0.15231429,  0.31644791,
          0.95022742,  1.07096768,  0.63536962, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          1.24686606,  1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     694.478000
1     850.190000
2     836.975000
3     961.431000
4     954.170000
5     886.165714
6     802.508571
7     730.261714
8     646.482000
9     485.012000
10    563.293000
11    744.863000
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    744.863
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.37406,-0.9372,-1.199219,-0.288358,0.55467,0.973144,0.602486,1.108405,2.635823,-0.189959,...,-2.260447,1.096579,0.927289,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.37406027, -0.93719981, -1.19921903, -0.28835757,
          0.55466994,  0.9731445 ,  0.60248617,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          1.09657936,  0

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    811.86
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(200)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2771472105, 87162449, 96536076, 197583216, 3094884934, 3152001847, 229051718, 3985062587, 296328490, 2593270571, 46090793, 2902606478, 2955128286, 1276754228, 2362842727, 4135900077, 4098399175, 2813993653, 3872563725, 3334043602, 337988486, 1562609188, 227060441, 3686393166, 1325815787, 3249496888, 1104299202, 3969322975, 2256155307, 1131999305, 3605035549, 3138888722, 3010804705, 800544984, 339845842, 1324909415, 984710172, 3271808189, 1221630551, 3767149043, 2511295260, 4232705098, 3185533721, 1254918623, 590509995, 4010339545, 1579767452, 3902433702, 1349467166, 547518288, 4097944408, 2579643584, 2163044047, 2498785350, 3588859333, 3508142177, 2358812247, 3940406399, 1105814572, 1202375715, 2210252457, 2304084794, 2066960777, 1402414590, 4094897743, 4143435618, 4182547372, 3972897713, 3090045624, 1413571728, 182209416, 3531640740, 3366332136, 1736829514, 1241302736, 2057932142, 2512224269, 1338536750, 3500764911, 2766883828, 3571504410, 2779516422, 450069757, 2496637455, 399291246

2023-09-13 07:01:31.842621: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-13 07:01:32.081559: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-13 07:01:32.081723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 102.61013793945312
winner_seed: 2771472105


Step: 1 ___________________________________________
loss: 112.90222930908203


Step: 2 ___________________________________________
loss: 264.0887451171875


Step: 3 ___________________________________________
loss: 257.7604064941406


Step: 4 ___________________________________________
loss: 318.9389343261719


Step: 5 ___________________________________________
loss: 166.25599670410156


Step: 6 ___________________________________________
loss: 79.9693832397461
winner_seed: 229051718


Step: 7 ___________________________________________
loss: 388.1572570800781


Step: 8 ___________________________________________
loss: 88.34868621826172


Step: 9 ___________________________________________
loss: 186.9458770751953


Step: 10 ___________________________________________
loss: 333.5593566894531


Step: 11 ___________________________________________
loss: 151.9078826904297


Step: 12 ___________________________________________
loss: 304.12

loss: 153.834716796875


Step: 104 ___________________________________________
loss: 353.0346374511719


Step: 105 ___________________________________________
loss: 407.8350524902344


Step: 106 ___________________________________________
loss: 457.8304138183594


Step: 107 ___________________________________________
loss: 178.94940185546875


Step: 108 ___________________________________________
loss: 197.29638671875


Step: 109 ___________________________________________
loss: 432.2276306152344


Step: 110 ___________________________________________
loss: 502.4539489746094


Step: 111 ___________________________________________
loss: 301.7444152832031


Step: 112 ___________________________________________
loss: 252.93125915527344


Step: 113 ___________________________________________
loss: 193.89598083496094


Step: 114 ___________________________________________
loss: 165.2022705078125


Step: 115 ___________________________________________
loss: 210.1098175048828


Step: 116 ____

In [21]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 602372.3125
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 588887.8750
Epoch 3/10000
1/1 [==============================] - 0s 8ms/step - loss: 549280.7500
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 257492.2656
Epoch 5/10000
1/1 [==============================] - 0s 7ms/step - loss: 34323.4102
Epoch 6/10000
1/1 [==============================] - 0s 8ms/step - loss: 17252.9766
Epoch 7/10000
1/1 [==============================] - 0s 8ms/step - loss: 9764.7822
Epoch 8/10000
1/1 [==============================] - 0s 8ms/step - loss: 8849.9688
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 8850.5098
Epoch 10/10000
1/1 [==============================] - 0s 8ms/step - loss: 7398.2915
Epoch 11/10000
1/1 [==============================] - 0s 8ms/step - loss: 6680.9336
Epoch 12/10000
1/1 [==============================] - 0s 5ms/step - loss: 79

1/1 [==============================] - 0s 6ms/step - loss: 3258.1882
Epoch 99/10000
1/1 [==============================] - 0s 6ms/step - loss: 1737.3976
Epoch 100/10000
1/1 [==============================] - 0s 6ms/step - loss: 4435.2261
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 2807.5137
Epoch 102/10000
1/1 [==============================] - 0s 6ms/step - loss: 1617.2765
Epoch 103/10000
1/1 [==============================] - 0s 6ms/step - loss: 2707.6069
Epoch 104/10000
1/1 [==============================] - 0s 5ms/step - loss: 2534.8704
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 3173.1003
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 3744.2959
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 3598.8943
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 2030.4741
Epoch 109/10000
1/1 [==============================] - 0s 5ms/step - loss: 2456.05

1/1 [==============================] - 0s 4ms/step - loss: 2009.7098
Epoch 195/10000
1/1 [==============================] - 0s 5ms/step - loss: 811.5845
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 1637.2817
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 1870.0430
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 1809.3867
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 1131.9775
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 1564.3795
Epoch 201/10000
1/1 [==============================] - 0s 5ms/step - loss: 1419.4176
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 2625.5417
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - loss: 2107.8372
Epoch 204/10000
1/1 [==============================] - 0s 5ms/step - loss: 1775.7285
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 758.392

1/1 [==============================] - 0s 5ms/step - loss: 581.3134
Epoch 291/10000
1/1 [==============================] - 0s 5ms/step - loss: 1718.5946
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 987.2790
Epoch 293/10000
1/1 [==============================] - 0s 5ms/step - loss: 1025.3138
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 1211.3258
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 1093.8773
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 1801.5746
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 1540.6810
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 1348.3695
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 2658.7244
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 1584.2289
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 2126.031

1/1 [==============================] - 0s 6ms/step - loss: 912.1794
Epoch 387/10000
1/1 [==============================] - 0s 6ms/step - loss: 727.1138
Epoch 388/10000
1/1 [==============================] - 0s 6ms/step - loss: 450.1215
Epoch 389/10000
1/1 [==============================] - 0s 6ms/step - loss: 1982.1952
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 1408.5009
Epoch 391/10000
1/1 [==============================] - 0s 6ms/step - loss: 453.3174
Epoch 392/10000
1/1 [==============================] - 0s 5ms/step - loss: 1583.5391
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 1770.6896
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 630.5458
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 1150.5670
Epoch 396/10000
1/1 [==============================] - 0s 5ms/step - loss: 1567.9127
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 1245.1034
E

...

Epoch 1444/10000
1/1 [==============================] - 0s 7ms/step - loss: 853.9771
Epoch 1445/10000
1/1 [==============================] - 0s 6ms/step - loss: 1624.6298
Epoch 1446/10000
1/1 [==============================] - 0s 6ms/step - loss: 1302.4393
Epoch 1447/10000
1/1 [==============================] - 0s 7ms/step - loss: 937.0118
Epoch 1448/10000
1/1 [==============================] - 0s 6ms/step - loss: 1433.4598
Epoch 1449/10000
1/1 [==============================] - 0s 7ms/step - loss: 624.7996
Epoch 1450/10000
1/1 [==============================] - 0s 5ms/step - loss: 528.5961
Epoch 1451/10000
1/1 [==============================] - 0s 7ms/step - loss: 824.8259
Epoch 1452/10000
1/1 [==============================] - 0s 5ms/step - loss: 855.6702
Epoch 1453/10000
1/1 [==============================] - 0s 7ms/step - loss: 1123.7943
Epoch 1454/10000
1/1 [==============================] - 0s 6ms/step - loss: 1865.9795
Epoch 1455/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 4ms/step - loss: 972.4023
Epoch 1540/10000
1/1 [==============================] - 0s 6ms/step - loss: 867.2072
Epoch 1541/10000
1/1 [==============================] - 0s 6ms/step - loss: 1308.6807
Epoch 1542/10000
1/1 [==============================] - 0s 6ms/step - loss: 977.1339
Epoch 1543/10000
1/1 [==============================] - 0s 6ms/step - loss: 790.4729
Epoch 1544/10000
1/1 [==============================] - 0s 6ms/step - loss: 974.8514
Epoch 1545/10000
1/1 [==============================] - 0s 4ms/step - loss: 758.5067
Epoch 1546/10000
1/1 [==============================] - 0s 5ms/step - loss: 731.3716
Epoch 1547/10000
1/1 [==============================] - 0s 4ms/step - loss: 1901.2432
Epoch 1548/10000
1/1 [==============================] - 0s 5ms/step - loss: 943.7684
Epoch 1549/10000
1/1 [==============================] - 0s 5ms/step - loss: 568.6459
Epoch 1550/10000
1/1 [==============================] - 0s 5ms/step - loss: 1094

1/1 [==============================] - 0s 5ms/step - loss: 1026.8593
Epoch 1636/10000
1/1 [==============================] - 0s 5ms/step - loss: 1871.5394
Epoch 1637/10000
1/1 [==============================] - 0s 5ms/step - loss: 612.4668
Epoch 1638/10000
1/1 [==============================] - 0s 5ms/step - loss: 1418.0231
Epoch 1639/10000
1/1 [==============================] - 0s 4ms/step - loss: 1160.9379
Epoch 1640/10000
1/1 [==============================] - 0s 4ms/step - loss: 1055.2955
Epoch 1641/10000
1/1 [==============================] - 0s 5ms/step - loss: 1573.4835
Epoch 1642/10000
1/1 [==============================] - 0s 6ms/step - loss: 687.7209
Epoch 1643/10000
1/1 [==============================] - 0s 6ms/step - loss: 1480.9218
Epoch 1644/10000
1/1 [==============================] - 0s 5ms/step - loss: 1311.7284
Epoch 1645/10000
1/1 [==============================] - 0s 5ms/step - loss: 1391.3046
Epoch 1646/10000
1/1 [==============================] - 0s 5ms/step - los

1/1 [==============================] - 0s 5ms/step - loss: 827.8068
Epoch 1731/10000
1/1 [==============================] - 0s 4ms/step - loss: 1026.0249
Epoch 1732/10000
1/1 [==============================] - 0s 5ms/step - loss: 255.4342
Epoch 1733/10000
1/1 [==============================] - 0s 5ms/step - loss: 642.4390
Epoch 1734/10000
1/1 [==============================] - 0s 5ms/step - loss: 1047.6952
Epoch 1735/10000
1/1 [==============================] - 0s 4ms/step - loss: 551.6404
Epoch 1736/10000
1/1 [==============================] - 0s 4ms/step - loss: 672.7339
Epoch 1737/10000
1/1 [==============================] - 0s 6ms/step - loss: 1812.9303
Epoch 1738/10000
1/1 [==============================] - 0s 6ms/step - loss: 2347.6531
Epoch 1739/10000
1/1 [==============================] - 0s 4ms/step - loss: 2225.2234
Epoch 1740/10000
1/1 [==============================] - 0s 4ms/step - loss: 1050.6487
Epoch 1741/10000
1/1 [==============================] - 0s 5ms/step - loss: 

Epoch 1826/10000
1/1 [==============================] - 0s 4ms/step - loss: 1061.7205
Epoch 1827/10000
1/1 [==============================] - 0s 4ms/step - loss: 945.1760
Epoch 1828/10000
1/1 [==============================] - 0s 4ms/step - loss: 1638.2920
Epoch 1829/10000
1/1 [==============================] - 0s 5ms/step - loss: 705.4960
Epoch 1830/10000
1/1 [==============================] - 0s 5ms/step - loss: 1011.1921
Epoch 1831/10000
1/1 [==============================] - 0s 5ms/step - loss: 497.4940
Epoch 1832/10000
1/1 [==============================] - 0s 5ms/step - loss: 736.9988
Epoch 1833/10000
1/1 [==============================] - 0s 4ms/step - loss: 1051.6743
Epoch 1834/10000
1/1 [==============================] - 0s 4ms/step - loss: 414.6163
Epoch 1835/10000
1/1 [==============================] - 0s 5ms/step - loss: 776.7029
Epoch 1836/10000
1/1 [==============================] - 0s 5ms/step - loss: 1331.0568
Epoch 1837/10000
1/1 [==============================] - 0s 4

In [22]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 618ms/step
[811.86](test_target) - [[746.95087]](prediction) = 64.90913330078126


In [23]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.0799511409612264

In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.920976,-1.384697,-0.378948,-0.630183,-1.282734,-1.243181,-0.462543,1.148205,-1.122043,-1.121418,...,1.412346,-1.260035,-1.258506,1.377871,1.395565,0.719114,0.276338,0.495192,0.906319,1.236047
1,-0.468948,0.443432,-0.990483,-0.781336,0.125669,0.037745,1.388657,0.140893,-0.184474,-0.185492,...,-0.643250,0.073929,0.070575,-0.413067,-0.896020,0.695031,1.062967,0.899613,0.487021,-0.022926
2,1.389925,0.941266,1.369431,1.411518,1.157065,1.205436,-0.926114,-1.289098,1.306518,1.306910,...,-0.769096,1.186106,1.187932,-0.964804,-0.499545,-1.414145,-1.339305,-1.394805,-1.393340,-1.213121


train_target:


,Rio Grande Do Norte - Consumo de Cimento (t)
0,694.478
1,850.190
2,836.975


test_input:


...

,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.37406,-0.9372,-1.199219,-0.288358,0.55467,0.973144,0.602486,1.108405,2.635823,-0.189959,...,-2.260447,1.096579,0.927289,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


test_target:


,Rio Grande Do Norte - Consumo de Cimento (t)
12,811.86


1/1 [==============================] - 1s 566ms/step
Error: 64.90913330078126




[834.3172607421875,
 947.6257934570312,
 962.599365234375,
 811.796142578125,
 806.10107421875,
 717.3458251953125,
 655.243896484375,
 501.0461120605469,
 545.1324462890625,
 746.9508666992188]

In [26]:
display(targets)
display(predictions)

[961.431,
 954.17,
 886.1657142857142,
 802.5085714285714,
 730.2617142857143,
 646.482,
 485.012,
 563.293,
 744.8629999999999,
 811.86]

[834.3172607421875,
 947.6257934570312,
 962.599365234375,
 811.796142578125,
 806.10107421875,
 717.3458251953125,
 655.243896484375,
 501.0461120605469,
 545.1324462890625,
 746.9508666992188]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

86.32008244628908

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.11378796156455274